In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ahmedhamada0/brain-tumor-detection", path="D:/Project Programes/pbl/kaggle/input")

print("Path to dataset files:", path)

path = kagglehub.dataset_download("migueldazbenito/json-notes", path="D:/Project Programes/pbl/kaggle/input")

print("Path to dataset files:", path)

In [ ]:
annotations_path = "/kaggle/input/json-notes/annotations_all_corregido.json"
images_path = "/kaggle/input/brain-tumor-detection/Br35H-Mask-RCNN/TRAIN/"
path = "/kaggle/input/brain-tumor-detection/Br35H-Mask-RCNN/"

In [ ]:
import os
import json
import cv2

def create_bbox_json(input_json, output_json):
    with open(input_json, 'r') as f:
        data = json.load(f)
    os.mkdir("D:/Project Programes/pbl/kaggle/working")
    os.mkdir("D:/Project Programes/pbl/kaggle/workingBr-35H-Mask-RCNN/")
    os.mkdir("D:/Project Programes/pbl/kaggle/working/Br-35H-Mask-RCNN/labels")
    new_data = []

    for image_info in data:
        new_image_info = {'filename': image_info, 'boxes': []}
        elemento = data[image_info]

        imagen = "D:/Project Programes/pbl/kaggle/input/brain-tumor-detection/yes/" +  image_info
        img = cv2.imread(imagen)
        height, width, channels = img.shape

        for region in elemento['regions']:
            box = {}
            shape_attributes = region['shape_attributes']
            region_attributes = region['region_attributes']
            box['class'] = '1'

            if 'all_points_x' in shape_attributes and 'all_points_y' in shape_attributes:
                # Obtener bounding box a partir de los puntos del polígono
                x_coords = shape_attributes['all_points_x']
                y_coords = shape_attributes['all_points_y']
                box['center_x'] = (min(x_coords) + max(x_coords)) / 2
                box['center_y'] = (min(y_coords) + max(y_coords)) / 2
                box['width'] = max(x_coords) - min(x_coords)
                box['height'] = max(y_coords) - min(y_coords)
            elif 'cx' in shape_attributes and 'cy' in shape_attributes and 'r' in shape_attributes:
                # Obtener bounding box a partir del centro y el radio del círculo
                center_x = shape_attributes['cx']
                center_y = shape_attributes['cy']
                radius = shape_attributes['r']
                box['center_x'] = center_x
                box['center_y'] = center_y
                box['width'] = 2 * radius
                box['height'] = 2 * radius
            elif 'cx' in shape_attributes and 'cy' in shape_attributes and 'rx' in shape_attributes and 'ry' in shape_attributes:
                # Obtener bounding box a partir del centro y los radios mayor y menor de la elipse
                center_x = shape_attributes['cx']
                center_y = shape_attributes['cy']
                minor_radius = shape_attributes['ry']
                major_radius = shape_attributes['rx']
                box['center_x'] = center_x
                box['center_y'] = center_y
                box['width'] = 2 * major_radius
                box['height'] = 2 * minor_radius

            # Normalizar las coordenadas
            box['center_x'] = round(box['center_x'] / width, 6)
            box['center_y'] = round(box['center_y'] / height, 6)
            box['width'] = round(box['width'] / width, 6)
            box['height'] = round(box['height'] / height, 6)

            new_image_info['boxes'].append(box)

        new_data.append(new_image_info)

        # Crear el archivo de texto para cada imagen
        txt_filename = os.path.join("D:/Project Programes/pbl/kaggle/working/Br-35H-Mask-RCNN/labels", os.path.splitext(image_info)[0] + '.txt')
        with open(txt_filename, 'w') as txt_file:
            for box in new_image_info['boxes']:
                txt_file.write(f"{box['class']} {box['center_x']} {box['center_y']} {box['width']} {box['height']}\n")

    with open(output_json, 'w') as f:
        json.dump(new_data, f, indent=2)

create_bbox_json("D:/Project Programes/pbl/kaggle/input/json-notes/annotations_all_corregido.json", 'D:/Project Programes/pbl/kaggle/working/Br-35H-Mask-RCNN/bbox.json')

In [ ]:
import shutil

# Definir rutas
train_path = 'D:/Project Programes/pbl/kaggle/working/Br-35H-Mask-RCNN/train/'
test_path = 'D:/Project Programes/pbl/kaggle/working/Br-35H-Mask-RCNN/test/'
images_path = "D:/Project Programes/pbl/kaggle/input/brain-tumor-detection/yes/"
annotations_path = "D:/Project Programes/pbl/kaggle/working/Br-35H-Mask-RCNN/labels/"

# Crear directorios si no existen
os.makedirs(train_path + 'images', exist_ok=True)
os.makedirs(train_path + 'labels', exist_ok=True)
os.makedirs(test_path + 'images', exist_ok=True)
os.makedirs(test_path + 'labels', exist_ok=True)

print('Copying Train Data..!!')
for i in range(0, 500):
    a = shutil.copyfile(images_path + 'y' + str(i) + '.jpg', train_path + 'images/y' + str(i) + '.jpg')
    a = shutil.copyfile(annotations_path + 'y' + str(i) + '.txt', train_path + 'labels/y' + str(i) + '.txt')

print('Copying Test Data..!!')
for i in range(701, 801):
    a = shutil.copyfile(images_path + 'y' + str(i) + '.jpg', test_path + 'images/y' + str(i) + '.jpg')
    a = shutil.copyfile(annotations_path + 'y' + str(i) + '.txt', test_path + 'labels/y' + str(i) + '.txt')